In [1]:
from bayes_opt import BayesianOptimization
from bayes_opt import SequentialDomainReductionTransformer
from bayes_opt import UtilityFunction
from dqn_scripts import *
from bayes_opt import SequentialDomainReductionTransformer
from bayes_opt.util import load_logs

In [2]:
trial = 0

In [3]:
def evaluate_and_test(params,trial_number,folder): #trial function that trains and stores results into array
    
    max_episodes = 20000
    
    #Train Env
    machine = Machine()
    machine.curr_state = 0
    env = MachineEnv(machine)

    #Eval Env
    machine2 = Machine()
    machine2.curr_state = 0
    env2 = MachineEnv(machine2)

    online_net = DoubleDQNet(4, 2)
    target_net = DoubleDQNet(4, 2)
    update_target_model(online_net, target_net)
    
    optimizer = optim.Adam(online_net.parameters(), lr=params[0])
    online_net.train()
    target_net.train()
    memory = Memory(10000)
    
    #start_time = datetime.now().replace(microsecond=0)

    running_score = 0
    epsilon = 1.0
    steps = 0
    loss = 0
    initial_exploration = 1000
    log_interval = 10
    
    tracker = [[0,0]]
    loss_arr = []
    max_average = 0
    
    for e in range(max_episodes):
        done = False

        score = 0
        state = env.reset()
        state = torch.Tensor(state)
        state = state.unsqueeze(0)
        while not done:
            steps += 1
            action = get_action(state, target_net, epsilon, env)
            next_state, reward, done, _ = env.step(action)

            next_state = torch.Tensor(next_state)
            next_state = next_state.unsqueeze(0)

            mask = 0 if done else 1
            #reward = reward if not done or score == 499 else -1
            action_one_hot = np.zeros(2)
            action_one_hot[action] = 1
            memory.push(state, next_state, action_one_hot, reward, mask)

            score += reward
            state = next_state

            if steps > initial_exploration:
                epsilon -= params[2]
                epsilon = max(epsilon, 0.1)

                batch = memory.sample(64)
                loss = DoubleDQNet.train_model(online_net, target_net, optimizer, batch,params[1])
                loss_arr.append(loss)
                
                if steps % params[3] == 0:
                    update_target_model(online_net, target_net)

        if e % log_interval == 0:
            if e == 0: continue
            eval_score = compute_avg_return(env2,online_net,20)
##            print('{} episode | Ave val score over 20 ep: {:.2f} | epsilon: {:.2f}'.format(
##                e, eval_score, epsilon))
            tracker.append([eval_score,e])
            
            move_ave_score = np.mean(np.array(tracker)[-50:,0])
##            if max_average < move_ave_score:
##                torch.save(online_net,f'{folder}/trial_{trial_number}_dqn_agent_.pt')
##                max_average = move_ave_score

##    np.savetxt(f'{folder}/trial_{trial_number}_loss.txt', loss_arr, delimiter=',',fmt='%s')
##    np.savetxt(f'{folder}/trial_{trial_number}.txt', tracker, delimiter=',',fmt='%s')
##    torch.save(online_net,f'{folder}/final_trial_{trial_number}_dqn_agent.pt')
    
    return (np.array(tracker),online_net)

In [5]:
def black_box_function(lr,gamma,eps_decay,update_freq):
    global trial
    trial+=1
    
    params = [lr,gamma,eps_decay,update_freq]
    
    results,policy = evaluate_and_test(params,trial,None)
    
#     score = np.mean(results[-500:])
    
    machine = Machine()
    machine.curr_state = 0
    env = MachineEnv(machine)
    
    score = compute_avg_return(env, policy, 10)
    
    return score

In [6]:
optimizer = BayesianOptimization(
    f = black_box_function,
    pbounds = {'lr':(1e-05,1e-01),'gamma':(0.8,0.99),'eps_decay':(1e-05,1e-01),'update_freq':(20,200)},
    verbose = 2,
)


df = pd.read_csv('results.csv')
df.drop('Unnamed: 0',axis = 1,inplace=True)
print(df)
df_arr = df.to_numpy()
df_arr

In [21]:
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events
logger = JSONLogger(path="./logs.json")
optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)

In [10]:
load_logs(optimizer, logs=["./logs.json"])

In [11]:
print(len(optimizer.space))

19


In [22]:
for row in df_arr:
    next_point_to_probe = {'lr':row[0],'gamma':row[1],'eps_decay':row[2],'update_freq':row[3]}
    target = row[7]
    optimizer.register(
     params=next_point_to_probe,
     target=target,
    )

|  5        |  8.514e+0 |  1e-05    |  0.8      |  0.0001   |  50.0     |
|  6        |  1.007e+0 |  0.0001   |  0.9      |  0.0001   |  100.0    |
|  7        |  2.663e+0 |  0.001    |  0.95     |  0.0001   |  150.0    |
|  8        |  2.054e+0 |  0.01     |  0.99     |  0.0001   |  200.0    |
|  9        |  9.65e+03 |  0.0001   |  0.8      |  0.001    |  150.0    |
|  10       |  1.222e+0 |  1e-05    |  0.9      |  0.001    |  200.0    |
|  11       |  1.09e+04 |  0.01     |  0.95     |  0.001    |  50.0     |
|  12       |  2.267e+0 |  0.001    |  0.99     |  0.001    |  100.0    |
|  13       |  9.953e+0 |  0.001    |  0.8      |  0.01     |  200.0    |
|  14       |  1.669e+0 |  0.01     |  0.9      |  0.01     |  150.0    |
|  15       |  1.276e+0 |  1e-05    |  0.95     |  0.01     |  100.0    |
|  16       |  2.501e+0 |  0.0001   |  0.99     |  0.01     |  50.0     |
|  17       |  1.045e+0 |  0.01     |  0.8      |  0.1      |  100.0    |
|  18       |  1.855e+0 |  0.001    | 

In [ ]:
optimizer.maximize(n_iter = 10,  init_points=1, acq='ei',xi=0.005 )

|   iter    |  target   | eps_decay |   gamma   |    lr     | update... |
-------------------------------------------------------------------------
|  1        |  5.66e+03 |  0.07609  |  0.9565   |  0.03884  |  74.66    |


In [24]:
optimizer._space._cache

{(0.09498021212209119,
  0.9377418778689102,
  0.07405646573726266,
  113.87312349042722): 5090.0,
 (0.08892742375350779,
  0.9050880308439506,
  0.05214285822278951,
  65.62488309842142): 2900.0,
 (0.034679306930964326,
  0.9300213042969555,
  0.055580584503608345,
  147.95783923378883): 8100.0,
 (0.02485776517901666,
  0.9077122605776904,
  0.07337076430335206,
  143.25063680580405): 4870.0,
 (1e-05, 0.8, 0.0001, 50.0): 8513.833333333334,
 (0.0001, 0.9, 0.0001, 100.0): 10067.213333333333,
 (0.001, 0.95, 0.0001, 150.0): 26630.37666666667,
 (0.01, 0.99, 0.0001, 200.0): 20541.68333333333,
 (0.0001, 0.8, 0.001, 150.0): 9649.66,
 (1e-05, 0.9, 0.001, 200.0): 12220.116666666667,
 (0.01, 0.95, 0.001, 50.0): 10904.936666666666,
 (0.001, 0.99, 0.001, 100.0): 22671.50666666667,
 (0.001, 0.8, 0.01, 200.0): 9953.073333333334,
 (0.01, 0.9, 0.01, 150.0): 16688.756666666668,
 (1e-05, 0.95, 0.01, 100.0): 12763.71,
 (0.0001, 0.99, 0.01, 50.0): 25010.763333333336,
 (0.01, 0.8, 0.1, 100.0): 10454.57,
 (

In [ ]:
utility = UtilityFunction(kind="ei",kappa= 0,xi=0)

In [ ]:
for _ in range(10):
    next_point = optimizer.suggest(utility)
    target = black_box_function(**next_point)
    optimizer.register(params=next_point, target=target)
    
    print(target, next_point)
print(optimizer.max)

In [ ]:
print(optimizer.max)

In [ ]:
for i, res in enumerate(optimizer.res):
    print("Iteration {}: \n\t{}".format(i, res))

In [ ]:
optimizer.maximize(
    init_points=2,
    n_iter=3,
    kernel = RBF(1.0)
)